# Stock Valuation using Free Cash Flow to the Firm with Python

**Hugh Donnelly, CFA**<br> 
*AlphaWave Data*

**November 2020**

For this analysis, we use several Python-based scientific computing technologies along with the [AlphaWave Data Financial Statements API](https://rapidapi.com/alphawave/api/financial-statements/endpoints) and the [AlphaWave Data Stock Analysis API](https://rapidapi.com/alphawave/api/stock-analysis/endpoints).  Jupyter Notebooks detailing this analysis are also available on [Google Colab](https://colab.research.google.com/drive/1BWmRIDtZGRndLk8lLGgpoURaAzHC8_jg?usp=sharing) and [Github](https://github.com/AlphaWaveData/Jupyter-Notebooks/blob/master/AlphaWave%20Stock%20Valuation%20using%20Free%20Cash%20Flow%20to%20the%20Firm%20example.ipynb).

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pandas_datareader as dr
import datetime

We will value Apple Inc. (AAPL) in the subsequent code; however, this analysis works with the ticker of any large publicly traded company.

In [2]:
company_ticker = 'AAPL'

### Introduction to Free Cash Flow Valuation

In stock valuation models, there are three predominant definitions of future cash flows: dividends, free cash flows, and residual income.  We will focus on building a free cash flow model for stock valuation in this article.  Let's discuss the advantages and disadvantages of free cash flow valuation.

One advantage of free cash flow models is that they can be applied to many firms, regardless of dividend policies or capital structures.  The ability to influence the distribution and application of a firm's free cash flow makes these models more pertinent to a firm's controlling shareholders.

However, there are cases in which the application of a free cash flow model may be very difficult.  Firms that have significant capital requirements may have negative free cash flow for many years into the future.  This can be caused by a technological revolution in an industry that requires greater investment to remain competitive or by rapid expansion into untapped markets.  This negative free cash flow complicates the cash flow forecast and makes the estimates less reliable.

Free cash flow models are most appropriate:
* For firms that do not have a dividend payment history or have a dividend payment history that is not clearly and appropriately related to earnings.
* For firms with free cash flow that corresponds with their profitability.
* When the valuation perspective is that of a controlling shareholder.

We will build our free cash flow model in three steps.  First, we will calculate the Free Cash Flow to the Firm.  Second, we will determine the Weighted Average Cost of Capital.  Finally, we will perform Stock Valuation using these inputs.

### 1. Free Cash Flow to the Firm

Forget about all the complicated financial statement relationships for a minute and simply picture the firm as a cash "processor."  Cash flows into the firm in the form of revenue as it sells its product, and cash flows out as it pays its cash operating expenses (e.g., salaries and taxes, but not interest expense, which is a financing and not an operating expense).  The firm takes the cash that is left over and makes short-term net investments in working capital (e.g., inventory and receivables) and long-term investments in property, plant, and equipment.  After these actions, the cash that remains is available to pay out to the firm's investors: bondholders, and common shareholders (let's assume for the moment that the firm has not issued preferred stock).  That pile of remaining cash is called Free Cash Flow to the Firm (FCFF) because it's "free" to payout to the firm's investors.

The formal definition of FCFF is the cash available to all of the firm's investors, including stockholders and bondholders, after the firm buys and sells products, provides services, pays its cash operating expenses, and makes short-term and long-term investments.

When building a FCFF valuation model, one can typically choose between a single-stage, two-stage, or three stage valuation model.  The single-stage FCFF valuation model is useful for stable firms in mature industries.  Two-stage and three-stage FCFF valuation models are used when you want to model multiple phases of growth (e.g., a growth phase, a mature phase, and a transition phase).  We use a two-stage FCFF valuation model in this example.

A general expression for the two-stage FCFF valuation model is:

$$Firm\ Value = \sum_{t=1}^n\overbrace{\frac{FCFF_t}{(1 + WACC)^t}}^{Stage\ 1}+\overbrace{\frac{FCFF_{n+1}}{(WACC - g)}*\frac{1}{(1 + WACC)^n}}^{Stage\ 2}$$<br>

$FCFF_t = expected\ free\ cash\ flow\ to\ the\ firm\ in\ year\ t$<br>
$FCFF_{n+1} = expected\ free\ cash\ flow\ to\ the\ firm\ in\ the\ last\ year\ (Year\ 6\ in\ this\ example)$<br>
$g = constant\ expected\ growth\ rate\ in\ FCFF$<br>
$WACC = weighted\ average\ cost\ of\ capital$<br>

One common two-stage model assumes a constant growth rate in each stage, and a second common model assumes declining growth in Stage 1 followed by a long-run sustainable growth rate in Stage 2.  In our model, we assume a constant growth rate in each stage.

We will show how to obtain each component of the two-stage FCFF valuation model, but we begin with expected FCFF which requires an ability to determine the appropriate growth rate.

To calculate the FCFF growth rate, we take the compound annual growth rate (CAGR) of FCFF over the past three years.  This FCFF growth rate is then used to generate FCFF forecasts for each of the next 5 years, which is Stage 1 in our model. For the constant expected growth rate that will be applied to Stage 2, we divide the CAGR of FCFF by two in order to estimate the long-term growth rate of FCFF in perpetuity.

In order to calculate the FCFF growth rate and long-term growth rate, we must first acquire the FCFF from the company's financial statements.  Using the [Cash Flow Statement](https://rapidapi.com/alphawave/api/financial-statements?endpoint=apiendpoint_1cf7a51e-32f9-41a5-901a-fa4f66ba9807) and [Income Statement](https://rapidapi.com/alphawave/api/financial-statements?endpoint=apiendpoint_b4be9cff-500e-4b0a-a5a9-a8d73f9cde7a) endpoints from the [AlphaWave Data Financial Statements API](https://rapidapi.com/alphawave/api/financial-statements/endpoints), we can access the data needed to calculate FCFF.

To call these APIs with Python, you can choose one of the supported Python code snippets provided in the API console. The following are examples of how to invoke the APIs with Python Requests. You will need to insert your own x-rapidapi-host and x-rapidapi-key information in the code blocks below.

In [3]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/cash-flow"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "YOUR_X-RAPIDAPI-HOST_WILL_COPY_DIRECTLY_FROM_RAPIDAPI_PYTHON_CODE_SNIPPETS",
    'x-rapidapi-key': "YOUR_X-RAPIDAPI-KEY_WILL_COPY_DIRECTLY_FROM_RAPIDAPI_PYTHON_CODE_SNIPPETS"
    }

cash_flow_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Cash Flow Statement DataFrame
cash_flow_df = pd.DataFrame.from_dict(cash_flow_response.json())
cash_flow_df = cash_flow_df.drop('ttm', axis = 1)
cash_flow_df

,9/30/2017,9/30/2018,9/30/2019,9/30/2020
Capital Expenditure,-12795000,-13313000,-10495000,-7309000
End Cash Position,20289000,25913000,50224000,39789000
Financing Cash Flow,-17347000,-87876000,-90976000,-86820000
Free Cash Flow,50803000,64121000,58896000,73365000
Income Tax Paid Supplemental Data,11591000,10417000,15263000,9501000
Interest Paid Supplemental Data,2092000,3022000,3423000,3002000
Investing Cash Flow,-46446000,16066000,45896000,-4289000
Issuance of Capital Stock,555000,669000,781000,880000
Issuance of Debt,28662000,6969000,6963000,16091000
Operating Cash Flow,63598000,77434000,69391000,80674000


In [4]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/income-statement"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "YOUR_X-RAPIDAPI-HOST_WILL_COPY_DIRECTLY_FROM_RAPIDAPI_PYTHON_CODE_SNIPPETS",
    'x-rapidapi-key': "YOUR_X-RAPIDAPI-KEY_WILL_COPY_DIRECTLY_FROM_RAPIDAPI_PYTHON_CODE_SNIPPETS"
    }

income_statement_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Income Statement DataFrame
income_statement_df = pd.DataFrame.from_dict(income_statement_response.json())
income_statement_df = income_statement_df.drop('ttm', axis = 1)
income_statement_df

,9/30/2017,9/30/2018,9/30/2019,9/30/2020
Basic Average Shares,20868968,19821508,18471336,17352119
Basic EPS,0.0023,0.003,0.003,0.0033
Cost of Revenue,141048000,163756000,161782000,169559000
Diluted Average Shares,21006768,20000436,18595652,17528214
Diluted EPS,0.0023,0.003,0.003,0.0033
Diluted NI Available to Com Stockholders,48351000,59531000,55256000,57411000
EBIT,66412000,76143000,69313000,69964000
Gross Profit,88186000,101839000,98392000,104956000
Interest Expense,2323000,3240000,3576000,2873000
Interest Income,5201000,5686000,4961000,3763000


Using the Cash Flow Statement and Income Statement, we calculate FCFF as:

$$FCFF = Free\ Cash\ Flow\ +\ Interest\ Expense*(1\ -\ Effective\ Tax\ Rate)$$

With this understanding, we are able to store the historical FCFF in a dataframe when running the code below.  

In [5]:
# FCFF Calculation using Cash Flow Statement and Income Statement Inputs
free_cash_flow_firm = (cash_flow_df.loc['Free Cash Flow'].astype(int) \
                    + (income_statement_df.loc['Interest Expense'].astype(int) \
                       * (1 - income_statement_df.loc['Tax Provision'].astype(int) \
                          / income_statement_df.loc['Pretax Income'].astype(int)))).astype(int)

# Change Series to a Pandas Dataframe
free_cash_flow_firm_df = free_cash_flow_firm.to_frame().transpose()
free_cash_flow_firm_df

,9/30/2017,9/30/2018,9/30/2019,9/30/2020
0,52555553,66766713,61901848,75823478


This allows us to proceed in determining the CAGR and the long-term growth rate that will be used in calculating the forecasted FCFF.

In [6]:
# CAGR of FCFF
latest_free_cash_flow_firm = float(free_cash_flow_firm_df.iloc[0,len(free_cash_flow_firm_df.columns)-1])
earliest_free_cash_flow_firm = float(free_cash_flow_firm_df.iloc[0,1])
free_cash_flow_firm_CAGR = (latest_free_cash_flow_firm/earliest_free_cash_flow_firm)\
                            **(float(1/(len(free_cash_flow_firm_df.columns)-2)))-1
free_cash_flow_firm_CAGR

0.06566782294550455

In [7]:
# Constant Expected Growth Rate
long_term_growth = free_cash_flow_firm_CAGR / 2
long_term_growth

0.032833911472752275

In [8]:
# Forecasted FCFF
forecast_free_cash_flow_firm_df = pd.DataFrame(columns=['Year ' + str(i) for i in range(1,7)])
free_cash_flow_firm_forecast_lst = []
for i in range(1,7):
    if i != 6:
        free_cash_flow_firm_forecast = latest_free_cash_flow_firm*(1+free_cash_flow_firm_CAGR)**i
    else:
        free_cash_flow_firm_forecast = latest_free_cash_flow_firm*(1+free_cash_flow_firm_CAGR)\
                                        **(i-1)*(1+long_term_growth)
    free_cash_flow_firm_forecast_lst.append(int(free_cash_flow_firm_forecast))
forecast_free_cash_flow_firm_df.loc[0] = free_cash_flow_firm_forecast_lst
forecast_free_cash_flow_firm_df

,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6
0,80802640,86108774,91763349,97789249,104210856,107632506


Now that we have successfully forecasted FCFF for the next six years and calculated the long-term growth rate, we turn our attention to WACC, which is the final input required in our two-stage FCFF valuation model.

### 2. Weighted Average Cost of Capital

The Weighted Average Cost of Capital (WACC) is the weighted average of the rates of return required by each of the capital suppliers to the firm (usually this is just equity and debt) where the weights are the proportions of the firm's total market value from each capital source:

WACC = (equity-to-capital ratio * required return on equity) + (debt-to-capital ratio * after-tax required return on debt)

$$WACC = \frac{E}{E+D}r_e\ +\ \frac{D}{E+D}r_d(1\ -\ tax\ rate)$$<br>
$E = market\ value\ of\ equity$<br>
$D = market\ value\ of\ debt$<br>
$r_e = required\ return\ on\ equity$<br>
$r_d = required\ return\ on\ debt$<br>
$tax\ rate = effective\ tax\ rate$<br>

Adding the [Balance Sheet Statement](https://rapidapi.com/alphawave/api/financial-statements?endpoint=apiendpoint_df251606-fb96-4468-9067-0d65886ce5b9) endpoint from the [AlphaWave Data Financial Statements API](https://rapidapi.com/alphawave/api/financial-statements/endpoints) and the [Key Statistics](https://rapidapi.com/alphawave/api/stock-analysis?endpoint=apiendpoint_dff4b882-4be4-4169-a700-04275c92bdce) endpoint from the [AlphaWave Data Stock Analysis API](https://rapidapi.com/alphawave/api/stock-analysis/endpoints), we can pull in the remaining data required to calculate WACC.

To call these APIs with Python, you can choose one of the supported Python code snippets provided in the API console. The following are examples of how to invoke the APIs with Python Requests. You will need to insert your own x-rapidapi-host and x-rapidapi-key information in the code blocks below.

In [9]:
url = "https://financial-statements.p.rapidapi.com/api/v1/resources/balance-sheet"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "YOUR_X-RAPIDAPI-HOST_WILL_COPY_DIRECTLY_FROM_RAPIDAPI_PYTHON_CODE_SNIPPETS",
    'x-rapidapi-key': "YOUR_X-RAPIDAPI-KEY_WILL_COPY_DIRECTLY_FROM_RAPIDAPI_PYTHON_CODE_SNIPPETS"
    }

balance_sheet_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Balance Sheet DataFrame
balance_sheet_df = pd.DataFrame.from_dict(balance_sheet_response.json())
balance_sheet_df

,9/30/2017,9/30/2018,9/30/2019,9/30/2020
Common Stock Equity,134047000,107147000,90488000,65339000
Invested Capital,249727000,221630000,198535000,177775000
Net Debt,95391000,88570000,59203000,74420000
Net Tangible Assets,126032000,107147000,90488000,65339000
Ordinary Shares Number,20504804,19019944,17772944,16976763
Share Issued,20504804,19019944,17772944,16976763
Tangible Book Value,126032000,107147000,90488000,65339000
Total Assets,375319000,365725000,338516000,323888000
Total Capitalization,231254000,200882000,182295000,164006000
Total Debt,115680000,114483000,108047000,112436000


In [10]:
url = "https://stock-analysis.p.rapidapi.com/api/v1/resources/key-stats"

querystring = {"ticker":company_ticker}

headers = {
    'x-rapidapi-host': "YOUR_X-RAPIDAPI-HOST_WILL_COPY_DIRECTLY_FROM_RAPIDAPI_PYTHON_CODE_SNIPPETS",
    'x-rapidapi-key': "YOUR_X-RAPIDAPI-KEY_WILL_COPY_DIRECTLY_FROM_RAPIDAPI_PYTHON_CODE_SNIPPETS"
    }

key_stats_response = requests.request("GET", url, headers=headers, params=querystring)

# Create Key Statistics DataFrame
key_stats_df = pd.DataFrame.from_dict(key_stats_response.json())
key_stats_df = key_stats_df.transpose()
key_stats_df

,Value
% held by insiders,0.07%
% held by institutions,60.63%
200-day moving average,101.94
5-year average dividend yield,1.50
50-day moving average,116.12
52-week change,78.60%
52-week high,137.98
52-week low,53.15
Avg vol (10-day),121.08M
Avg vol (3-month),158.01M




**2a. Estimating the Required Return on Equity**<br>
In order to calculate the Required Return on Equity, we will use the Capital Asset Pricing Model (CAPM) which is an equilibrium model that takes the risk-free rate, the stock's equity beta, and the market risk premium as inputs.

$$r_e = r_f\ +(\beta*MRP)$$<br>
$r_e = required\ return\ on\ equity$<br>
$r_f = risk-free\ rate$<br>
$\beta = beta$<br>
$MRP = market\ risk\ premium$<br>

*Estimating the Risk-Free Rate*<br>
Government securities are assumed to be risk-free, at least from a credit standpoint.  With this assumption, the appropriate rate to use in the CAPM is the government security having approximately the same duration as the asset being valued and sufficient liquidity so that the yield does not have an embedded liquidity risk premium.  Equities are assumed to have a long duration, so a long-term government bond yield is an appropriate proxy for the risk-free rate.

In this model, the yield on the 10 Year U.S. Treasury Note will be used as the risk-free rate.  We can scrape the current yield on the 10 Year U.S. Treasury Note from Yahoo Finance using the code below.

In [11]:
# Risk-free Rate
timespan = 100
current_date = datetime.date.today()
past_date = current_date-datetime.timedelta(days=timespan)
risk_free_rate_df = dr.DataReader('^TNX', 'yahoo', past_date, current_date) 
risk_free_rate = (risk_free_rate_df.iloc[len(risk_free_rate_df)-1,5])/100
risk_free_rate

0.008930000066757203

*Estimating the Stock's Beta*<br>
A stock's beta is the sensitivity of that stock's return to the return of the market index.  It is a measure of systematic risk and can be obtained using the [Key Statistics](https://rapidapi.com/alphawave/api/stock-analysis?endpoint=apiendpoint_dff4b882-4be4-4169-a700-04275c92bdce) endpoint from the [AlphaWave Data Stock Analysis API](https://rapidapi.com/alphawave/api/stock-analysis/endpoints) using the code below.

In [12]:
# Stock's Beta
equity_beta = float(key_stats_df.loc[r'Beta (5Y monthly)'])
equity_beta

1.35

*Estimating the Market Risk Premium*<br>
The market risk premium should be the expected return on the market index less the expected return (or yield) on the long-term government bond.  For our purposes, we use the annual [market risk premium](http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html) provided by Aswath Damodaran, who is a professor at the Stern School of Business at New York University.  Professor Damodaran teaches Corporate Finance and Equity Instruments & Markets.  He provides a multitude of financial information and I encourage you to visit his [website](http://pages.stern.nyu.edu/~adamodar/).

In [13]:
# Market Risk Premium
market_risk_premium = 0.0523

Using the CAPM, we can now calculate the required return on equity for the company.

In [14]:
# Required Return on Equity
equity_return = risk_free_rate + (equity_beta*market_risk_premium)
equity_return

0.0795350000667572

**2b. Estimating the Required Return on Debt**<br>
Firms have a cost of debt, which is the interest rate a company pays on its debt, such as bonds and loans. From the lender's perspective, this rate is the required return on debt. To calculate the required return on debt, we use the [AlphaWave Data Financial Statements API](https://rapidapi.com/alphawave/api/financial-statements/endpoints) to pull the firm's Interest Expense from the [Income Statement](https://rapidapi.com/alphawave/api/financial-statements?endpoint=apiendpoint_b4be9cff-500e-4b0a-a5a9-a8d73f9cde7a) endpoint and Total Debt from the [Balance Sheet Statement](https://rapidapi.com/alphawave/api/financial-statements?endpoint=apiendpoint_df251606-fb96-4468-9067-0d65886ce5b9) endpoint.

After dividing Interest Expense by Total Debt to calculate the required rate of return on debt, we will multiply this rate by (1 - effective tax rate) as seen in the WACC formula in order to get the after-tax required return on debt.  The reason for this is because there are tax deductions available on interest paid, which are often to the firm's benefit. As a result, the net cost of a company's debt is the amount of interest the firm is paying minus the amount it has saved in taxes because of its tax-deductible interest payments.

Using the [AlphaWave Data Financial Statements API](https://rapidapi.com/alphawave/api/financial-statements/endpoints), we can calculate the effective tax rate by taking the average of the Tax Provision values returned through the [Income Statement](https://rapidapi.com/alphawave/api/financial-statements?endpoint=apiendpoint_b4be9cff-500e-4b0a-a5a9-a8d73f9cde7a) endpoint.

The code below gives a good approximation for the required rate of return on debt and the effective tax rate that will be used in the WACC calculation.

In [15]:
# Interest Expense
interest_expense = income_statement_df.loc['Interest Expense']
interest_expense_df = interest_expense.to_frame().transpose()
interest_expense_str = interest_expense_df.values[0][-1:]
interest_expense_int = int(interest_expense_str)

# Total Debt
total_debt = balance_sheet_df.loc['Total Debt']
total_debt_df = total_debt.to_frame().transpose()
total_debt_str = total_debt_df.values[0][-1:]
total_debt_int = int(total_debt_str)

# Required Return on Debt
debt_return = interest_expense_int / total_debt_int
debt_return

0.025552314205414636

In [16]:
# Effective Tax Rate
effective_tax_rate = income_statement_df.loc['Tax Provision'].astype(int) \
                    / income_statement_df.loc['Pretax Income'].astype(int)
avg_effective_tax_rate = sum(effective_tax_rate) / len(effective_tax_rate)
avg_effective_tax_rate

0.18317664597985506

**2c. Estimating the Capital Structure**<br>
The final inputs for calculating WACC deal with the firm's capital structure, which is a company's mix of equity and debt.  As previously noted, WACC is a blend of a company’s equity and debt cost of capital based on the company's equity and debt capital ratio.

To calculate the market value of equity for a firm, we can use the [AlphaWave Data Stock Analysis API](https://rapidapi.com/alphawave/api/stock-analysis/endpoints) to pull the firm's Market Cap from the [Key Statistics](https://rapidapi.com/alphawave/api/stock-analysis?endpoint=apiendpoint_dff4b882-4be4-4169-a700-04275c92bdce) endpoint.

In [17]:
# Market Value of Equity
market_cap_str = key_stats_df.loc[r'Market cap (intra-day) '][0]

market_cap_lst = market_cap_str.split('.')
if market_cap_str[len(market_cap_str)-1] == 'T':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('T',(9-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))
if market_cap_str[len(market_cap_str)-1] == 'B':
    market_cap_length = len(market_cap_lst[1])-1
    market_cap_lst[1] = market_cap_lst[1].replace('B',(6-market_cap_length)*'0')
    market_cap_int = int(''.join(market_cap_lst))

market_cap_int

2030000000

In most cases, you can use the book value of debt from a company’s latest balance sheet as an approximation for market value of debt. Unlike equity, the market value of debt usually does not deviate too far from the book value.  To calculate the market value of debt, we can use the [AlphaWave Data Financial Statements API](https://rapidapi.com/alphawave/api/financial-statements/endpoints) to pull the firm's Net Debt from the [Balance Sheet Statement](https://rapidapi.com/alphawave/api/financial-statements?endpoint=apiendpoint_df251606-fb96-4468-9067-0d65886ce5b9) endpoint.

In [18]:
# Market Value of Debt
net_debt = balance_sheet_df.loc['Net Debt']
net_debt_df = net_debt.to_frame().transpose()
net_debt_str = net_debt_df.values[0][-1:]
net_debt_int = int(net_debt_str)
net_debt_int

74420000

The company value is the sum of its equity and debt market values.

In [19]:
# Company Value
company_value = market_cap_int + net_debt_int
company_value

2104420000

We are now able to determine the equity and debt capital ratios by dividing both the market value of equity and the market value of debt by the company value.

With all the components acquired, we can calculate the firm's WACC.

In [20]:
WACC = ((market_cap_int/company_value) * equity_return) \
        + ((net_debt_int/company_value) * (debt_return * (1-avg_effective_tax_rate)))
WACC

0.07746045183864433

### 3. Stock Valuation

We need to discount all the forecasted cash flows to get the Firm Value. First, discount the forecasted FCFF for each of the next five years (Stage 1). Then, the present value of the terminal value of the firm using the forecasted FCFF in Year 6 is added to this amount (Stage 2).

$$Firm\ Value = \sum_{t=1}^n\overbrace{\frac{FCFF_t}{(1 + WACC)^t}}^{Stage\ 1}+\overbrace{\frac{FCFF_{n+1}}{(WACC - g)}*\frac{1}{(1 + WACC)^n}}^{Stage\ 2}$$<br>

$FCFF_t = expected\ free\ cash\ flow\ to\ the\ firm\ in\ year\ t$<br>
$FCFF_{n+1} = expected\ free\ cash\ flow\ to\ the\ firm\ in\ the\ last\ year\ (Year\ 6\ in\ this\ example)$<br>
$g = constant\ expected\ growth\ rate\ in\ FCFF$<br>
$WACC = weighted\ average\ cost\ of\ capital$<br>

As the resulting value gives us the Firm Value of the company (which is the value to both equity and debt holders), we need to deduct the company’s Market Value of Debt to arrive at the Equity Value of the firm.

Equity Value = Firm Value – Market Value of Debt

In [21]:
# Equity Value Calculation
discounted_FCFF_lst = []
for year in range(0,5):
    discounted_FCFF = forecast_free_cash_flow_firm_df.iloc[0,year]/(1+WACC)**(year+1)
    discounted_FCFF_lst.append(int(discounted_FCFF))
terminal_value = forecast_free_cash_flow_firm_df.iloc[0,5]/(WACC-long_term_growth)
PV_terminal_value = int(terminal_value/(1+WACC)**5)
enterprise_value = sum(discounted_FCFF_lst)+PV_terminal_value
equity_value = enterprise_value-net_debt_int
equity_value

1953329634

The [Key Statistics](https://rapidapi.com/alphawave/api/stock-analysis?endpoint=apiendpoint_dff4b882-4be4-4169-a700-04275c92bdce) endpoint from the [AlphaWave Data Stock Analysis API](https://rapidapi.com/alphawave/api/stock-analysis/endpoints) enables us to pull in the firm's total Shares Outstanding, which we will use to calculate the two-stage FCFF valuation model's stock price estimate for the firm.

In [22]:
# Total Shares Outstanding
shares_outstanding_str = key_stats_df.loc[r'Shares outstanding '][0]

shares_outstanding_lst = shares_outstanding_str.split('.')
if shares_outstanding_str[len(shares_outstanding_str)-1] == 'T':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('T',(9-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))
if shares_outstanding_str[len(shares_outstanding_str)-1] == 'B':
    shares_outstanding_length = len(shares_outstanding_lst[1])-1
    shares_outstanding_lst[1] = shares_outstanding_lst[1].replace('B',(6-shares_outstanding_length)*'0')
    shares_outstanding_int = int(''.join(shares_outstanding_lst))

shares_outstanding_int

17100000

In [23]:
# Two-stage FCFF Valuation Model Stock Price Estimate
stock_price = equity_value / shares_outstanding_int
stock_price = '${:,.2f}'.format(stock_price)
print("Model Stock Price = %s"%(stock_price))

# Actual Stock Price
actual_stock_price = market_cap_int / shares_outstanding_int
actual_stock_price = '${:,.2f}'.format(actual_stock_price)
print("Actual Stock Price = %s"%(actual_stock_price))

Model Stock Price = $114.23
Actual Stock Price = $118.71


### 4. Conclusion

As can be seen in this article, we were able to calculate Stock Valuation using FCFF and WACC.  In the example above, the two-stage FCFF valuation model does a reasonably good job at estimating the fair value of a company's stock price.  However, the model is very sensitive to the discount and growth rates selected.  When applying this model, it is important to consider the type of growth the company is currently experiencing along with the anticipated growth characteristics for the future. A single-stage FCFF valuation model is useful for stable firms in mature industries while two-stage and three-stage FCFF valuation models are useful when you want to model multiple phases of growth.

If you liked this article, you may also enjoy AlphaWave Data's [Bayesian Pairs Trading using Corporate Supply Chain Data](https://medium.com/@hdonnelly6/bayesian-pairs-trading-using-corporate-supply-chain-data-8b96305686d) article.  Using the [AlphaWave Data Corporate Supply Chain API](https://rapidapi.com/alphawave/api/corporate-supply-chain/endpoints), we can get a list of suppliers and customers for a given stock symbol.  You can further filter this list of suppliers and customers using the two-stage FCFF valuation model discussed in this article to get a grouping of good pairs trading candidates that are potentially misvalued.  A list of company tickers that have supply chain data is included at the bottom of the Jupyter Notebooks available on [Google Colab](https://colab.research.google.com/drive/1e_SiiZn7WEW3OUNG-ftN3riPNjiz0M0C?usp=sharing) and [Github](https://github.com/AlphaWaveData/Jupyter-Notebooks/blob/master/AlphaWave%20Corporate%20Supply%20Chain%20API%20Example.ipynb), which could be a good list of tickers to run through the two-stage FCFF valuation model discussed above.